In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb

#path.join(path.dirname(os.getcwd()), "credentials.ini")

creds_path_ar = ["../../credentials.ini","credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [3]:

    wandb.init(project="3dcnn")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[HR]conv3d'

&lt;IPython.core.display.HTML object&gt;

In [4]:
cd {PATH_ROOT}

/home/ubuntu/sota-mafat-radar


In [5]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data, get_data_pipeline
from src.models import arch_setup, dataset_ram_reduced, base_base_model, alex_model, base_3d
from src.features import specto_feat

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [6]:
config = dict()
config['num_tracks'] = 3
config['val_ratio'] = 3
config['shift_segment'] = [25]
config['get_shifts'] = False
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False
config['wavelets'] = True

batch_size = 32
lr = 1e-4

In [36]:
full_data = get_data_pipeline.pipeline_trainval_ram_reduced(PATH_DATA, config)

adding auxilary start
100%|██████████| 6877/6877 [21:05&lt;00:00,  5.44it/s]


In [39]:
#full_data.to_csv('data/full_data.csv')

In [38]:
#full_data = pd.read_csv('data/full_data.csv',index_col=0)
full_data['scalogram'][0].shape

(126, 32, 8)

In [40]:
train = full_data[full_data.is_validation == False]
validation = full_data[full_data.is_validation == True]

print(train.shape[0])
print(validation.shape[0])

6261
616


In [49]:
train_set = dataset_ram_reduced.DS(train,'scalogram')
val_set= dataset_ram_reduced.DS(validation,'scalogram')

train_y = np.array(full_data[full_data.is_validation==False]['target_type']).astype(int)
val_y = np.array(full_data[full_data.is_validation==True]['target_type']).astype(int)

train_loader=DataLoader(dataset= train_set, batch_size = batch_size, shuffle = True, num_workers = 32)
val_loader=DataLoader(dataset= val_set, batch_size = batch_size, shuffle = True, num_workers = 32)

In [43]:
model= base_3d.alex_3d()
# model.apply(init_weights)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)

if WANDB_enable == True:
    runname = input("Enter WANDB runname(ENTER to skip wandb) :")
    notes = input("Enter run notes :")

    wandb.init(project="3dcnn",name=runname, notes=notes)
    os.environ['WANDB_NOTEBOOK_NAME'] = '[HR]conv3d'
    
    wandb.watch(model)
    wandb.config['data_config'] = config
    wandb.config['train_size'] = train.shape[0]
    wandb.config['val_size'] = validation.shape[0]
    wandb.config['batch_size'] = batch_size
    wandb.config['learning rate'] = lr
    wandb.log(config)


&lt;IPython.core.display.HTML object&gt;

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.10.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [50]:
np.unique(train_y)

array([0, 1])

In [47]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

started training epoch no. 1


RuntimeError: number of dims don&#39;t match in permute

In [ ]:
arch_setup.plot_loss_train_test(log,model)

In [ ]:
arch_setup.plot_ROC_local_gpu(train_loader,val_loader,model,device)

In [31]:
importlib.reload(arch_setup)

<module 'src.models.arch_setup' from '/home/ubuntu/sota-mafat-radar/src/models/arch_setup.py'>

In [34]:
full_data.augmentation_info.apply(lambda x: len(x[0]))

0       1
1       1
2       1
3       1
4       1
       ..
6872    1
6873    1
6874    1
6875    1
6876    1
Name: augmentation_info, Length: 6877, dtype: int64